# `transformers` meets `bitsandbytes` for democratzing Large Language Models (LLMs) through 4bit quantization

<center>
<img src="https://github.com/huggingface/blog/blob/main/assets/96_hf_bitsandbytes_integration/Thumbnail_blue.png?raw=true" alt="drawing" width="700" class="center"/>
</center>

Welcome to this notebook that goes through the recent `bitsandbytes` integration that includes the work from XXX that introduces no performance degradation 4bit quantization techniques, for democratizing LLMs inference and training.

In this notebook, we will learn together how to load a large model in 4bit (`gpt-neo-x-20b`) and train it using Google Colab and PEFT library from Hugging Face 🤗.

[In the general usage notebook](https://colab.research.google.com/drive/1ge2F1QSK8Q7h0hn3YKuBCOAS0bK8E0wf?usp=sharing), you can learn how to propely load a model in 4bit with all its variants. 

If you liked the previous work for integrating [*LLM.int8*](https://arxiv.org/abs/2208.07339), you can have a look at the [introduction blogpost](https://huggingface.co/blog/hf-bitsandbytes-integration) to lean more about that quantization method.


In [7]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun May 28 07:35:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10          On   | 00000000:06:00.0 Off |                    0 |
|  0%   47C    P0    61W / 150W |   9678MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# !pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git 
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
# !pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.2/198.2 KB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 126.0 MB/s eta 0:00:00


In [11]:
!pip install -r ../requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-zyu6s71a
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-zyu6s71a
  Resolved https://github.com/huggingface/transformers.git to commit 17a55534f5e5df10ac4804d4270bf6b8cc24998d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-al8zgno_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-al8zgno_
  Resolved https://github.com/huggingface/peft.git to commit 3714aa2fff158fdfa637b2b65952580801d890b2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https:

First let's load the model we are going to use - GPT-neo-x-20B! Note that the model itself is around 40GB in half precision

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [13]:
# model_id = "EleutherAI/gpt-neox-20b"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Loading checkpoint shards:  83%|████████▎ | 38/46 [00:09<00:02,  3.95it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 288.00 MiB (GPU 0; 22.19 GiB total capacity; 20.97 GiB already allocated; 88.50 MiB free; 21.29 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [4]:
# https://huggingface.co/tiiuae/falcon-7b
# !pip install einops

"""
model_id = "tiiuae/falcon-7b"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, trust_remote_code=True)
"""

'\nmodel_id = "tiiuae/falcon-7b"\n\ntokenizer = AutoTokenizer.from_pretrained(model_id)\nmodel = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, trust_remote_code=True)\n'

In [4]:
# !pip install -q -U sentencepiece

from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, LlamaTokenizerFast, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer

# model_id = "decapoda-research/llama-7b-hf"
model_id = "huggyllama/llama-7b"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    #load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    device_map={"": 0}
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
# tokenizer = LlamaTokenizer.from_pretrained(model_id)
# tokenizer = LlamaTokenizer.from_pretrained(model_id)
tokenizer.bos_token_id = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Then we have to apply some preprocessing to the model to prepare it for training. For that use the `prepare_model_for_kbit_training` method from PEFT.

In [5]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [6]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [9]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    target_modules=["query_key_value"], 
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 12>:12                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/peft/mapping.py:120 in get_peft_model                    │
│                                                                                                  │
│   117 │   │   return PeftModel(model, peft_config)                                               │
│   118 │   if isinstance(peft_config, PromptLearningConfig):                                      │
│   119 │   │   peft_config = _prepare_prompt_learning_config(peft_config, model_config)           │
│ ❱ 120 │   return MODEL_TYPE_TO_PEFT_MODEL_MAPPING[peft_config.task_type](model, peft_config)     │
│   121                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/peft/peft_model.py:670 in __init__                       │
│                                                                                                  │
│    667 │   """                                                                                   │
│    668 │                                                                                         │
│    669 │   def __init__(self, model, peft_config: PeftConfig, adapter_name="default"):           │
│ ❱  670 │   │   super().__init__(model, peft_config, adapter_name)                                │
│    671 │   │   self.base_model_prepare_inputs_for_generation = self.base_model.prepare_inputs_f  │
│    672 │                                                                                         │
│    673 │   def forward(                                                                          │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/peft/peft_model.py:99 in __init__                        │
│                                                                                                  │
│     96 │   │   self.base_model_torch_dtype = getattr(model, "dtype", None)                       │
│     97 │   │   if not isinstance(peft_config, PromptLearningConfig):                             │
│     98 │   │   │   self.peft_config[adapter_name] = peft_config                                  │
│ ❱   99 │   │   │   self.base_model = PEFT_TYPE_TO_MODEL_MAPPING[peft_config.peft_type](          │
│    100 │   │   │   │   self.base_model, self.peft_config, adapter_name                           │
│    101 │   │   │   )                                                                             │
│    102 │   │   │   self.set_additional_trainable_modules(peft_config, adapter_name)              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/peft/tuners/lora.py:154 in __init__                      │
│                                                                                                  │
│   151 │   │   self.model = model                                                                 │
│   152 │   │   self.forward = self.model.forward                                                  │
│   153 │   │   self.peft_config = config                                                          │
│ ❱ 154 │   │   self.add_adapter(adapter_name, self.peft_config[adapter_name])                     │
│   155 │                                                                                          │
│   156 │   def add_adapter(self, adapter_name, config=None):                                      │
│   157 │   │   if config is not None:                       

Let's load a common dataset, english quotes, to fine tune our model on famous quotes.

In [47]:
from datasets import load_dataset

data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)

  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [48]:
print(data)

DatasetDict({
    train: Dataset({
        features: ['quote', 'author', 'tags', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2508
    })
})


Run the cell below to run the training! For the sake of the demo, we just ran it for few steps just to showcase how to use this integration with existing tools on the HF ecosystem.

In [53]:
output_path = "outputs-2"

In [57]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=10,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir=output_path,
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 23>:23                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1696 in train                    │
│                                                                                                  │
│   1693 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1694 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1695 │   │   )                                                                                 │
│ ❱ 1696 │   │   return inner_training_loop(                                                       │
│   1697 │   │   │   args=args,                                                                    │
│   1698 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1699 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2035 in _inner_training_loop     │
│                                                                                                  │
│   2032 │   │   │   │   │   │   │   xm.optimizer_step(self.optimizer)                             │
│   2033 │   │   │   │   │   elif self.do_grad_scaling:                                            │
│   2034 │   │   │   │   │   │   scale_before = self.scaler.get_scale()                            │
│ ❱ 2035 │   │   │   │   │   │   self.scaler.step(self.optimizer)                                  │
│   2036 │   │   │   │   │   │   self.scaler.update()                                              │
│   2037 │   │   │   │   │   │   scale_after = self.scaler.get_scale()                             │
│   2038 │   │   │   │   │   │   optimizer_was_run = scale_before <= scale_after                   │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/cuda/amp/grad_scaler.py:372 in step                │
│                                                                                                  │
│   369 │   │   if optimizer_state["stage"] is OptState.READY:                                     │
│   370 │   │   │   self.unscale_(optimizer)                                                       │
│   371 │   │                                                                                      │
│ ❱ 372 │   │   assert len(optimizer_state["found_inf_per_device"]) > 0, "No inf checks were rec   │
│   373 │   │                                                                                      │
│   374 │   │   retval = self._maybe_opt_step(optimizer, optimizer_state, *args, **kwargs)         │
│   375                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AssertionError: No inf checks were recorded for this optimizer.

In [50]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_path)

In [51]:
lora_config = LoraConfig.from_pretrained(output_path)
model = get_peft_model(model, lora_config)

In [52]:
text = "Elon Musk "
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 5>:5                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/peft/peft_model.py:743 in generate                       │
│                                                                                                  │
│    740 │   │   │   self.base_model.generation_config = self.generation_config                    │
│    741 │   │   try:                                                                              │
│    742 │   │   │   if not isinstance(peft_config, PromptLearningConfig):                         │
│ ❱  743 │   │   │   │   outputs = self.base_model.generate(**kwargs)                              │
│    744 │   │   │   else:                                                                         │
│    745 │   │   │   │   if "input_ids" not in kwargs:                                             │
│    746 │   │   │   │   │   raise ValueError("input_ids must be provided for Peft model generati  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/peft/peft_model.py:743 in generate                       │
│                                                                                                  │
│    740 │   │   │   self.base_model.generation_config = self.generation_config                    │
│    741 │   │   try:                                                                              │
│    742 │   │   │   if not isinstance(peft_config, PromptLearningConfig):                         │
│ ❱  743 │   │   │   │   outputs = self.base_model.generate(**kwargs)                              │
│    744 │   │   │   else:                                                                         │
│    745 │   │   │   │   if "input_ids" not in kwargs:                                             │
│    746 │   │   │   │   │   raise ValueError("input_ids must be provided for Peft model generati  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py:115 in decorate_context       │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1267 in generate        │
│                                                                                                  │
│   1264 │   │   generation_config = copy.deepcopy(generation_config)                              │
│   1265 │   │   model_kwargs = generation_config.update(**kwargs)  # All unused kwargs must be m  │
│   1266 │   │   generation_config.validate()                                                      │
│ ❱ 1267 │   │   self._validate_model_kwargs(model_kwargs.copy())                                  │
│   1268 │   │                                               

In [18]:
from transformers import TextStreamer

# inputs = tokenizer(["An increasing sequence: one,"], return_tensors="pt")
streamer = TextStreamer(tokenizer)

# Despite returning the usual output, the streamer will also print the generated text to stdout.
outputs = model.generate(**inputs, streamer=streamer, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Elon Musk 
Elon Musk is the founder of SpaceX and the CEO of Tesla, Inc.


Elon Musk 
Elon Musk is the founder of SpaceX and the CEO of Tesla, Inc.




In [22]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install -q -U guidance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.8 MB/s eta 0:00:00


In [25]:
import guidance

# set the default language model used to execute guidance programs
guidance.llm = guidance.llms.Transformers(model=model, tokenizer=tokenizer, device=0)

ERROR:asyncio:Task was destroyed but it is pending!
task: <Task pending name='Task-1' coro=<DisplayThrottler.run() running at /usr/local/lib/python3.10/dist-packages/guidance/_program.py:665> wait_for=<Future pending cb=[Task.__wakeup()]>>


In [ ]:
# define a guidance program that adapts a proverb
program = guidance("""Tweak this proverb to apply to model instructions instead.

{{proverb}}
- {{book}} {{chapter}}:{{verse}}

UPDATED
Where there is no guidance{{gen 'rewrite' stop="\\n-"}}
- GPT {{gen 'chapter'}}:{{gen 'verse'}}""")

# execute the program on a specific proverb
executed_program = program(
    proverb="Where there is no guidance, a people falls,\nbut in an abundance of counselors there is safety.",
    book="Proverbs",
    chapter=11,
    verse=14
)
print(executed_program)

In [33]:
# define a guidance program that adapts a proverb
program = guidance("""{{person}} is{{gen 'description' stop="."}}""")

# execute the program on a specific proverb
executed_program = program(
  person="Elon Musk"
)
print(executed_program)

Stop program Elon Musk is

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/guidance/_program_executor.py", line 94, in run
    await self.visit(self.parse_tree)
  File "/usr/local/lib/python3.10/dist-packages/guidance/_program_executor.py", line 429, in visit
    visited_children.append(await self.visit(child, inner_next_node, inner_next_next_node, inner_prev_node, node, parent_node))
  File "/usr/local/lib/python3.10/dist-packages/guidance/_program_executor.py", line 429, in visit
    visited_children.append(await self.visit(child, inner_next_node, inner_next_next_node, inner_prev_node, node, parent_node))
  File "/usr/local/lib/python3.10/dist-packages/guidance/_program_executor.py", line 218, in visit
    visited_children = [await self.visit(child, next_node, next_next_node, prev_node, node, parent_node) for child in node.children]
  File "/usr/local/lib/python3.10/dist-packages/guidance/_program_executor.py", line 218, in <listcomp>
    visited_children = [await self.visit(

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 5>:5                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/guidance/_program.py:233 in __call__                     │
│                                                                                                  │
│   230 │   │   │   if new_program.stream:                                                         │
│   231 │   │   │   │   return self._stream_run(loop, new_program)                                 │
│   232 │   │   │   else:                                                                          │
│ ❱ 233 │   │   │   │   loop.run_until_complete(new_program.execute())                             │
│   234 │   │                                                                                      │
│   235 │   │   return new_program                                                                 │
│   236                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/nest_asyncio.py:90 in run_until_complete                 │
│                                                                                                  │
│    87 │   │   │   if not f.done():                                                               │
│    88 │   │   │   │   raise RuntimeError(                                                        │
│    89 │   │   │   │   │   'Event loop stopped before Future completed.')                         │
│ ❱  90 │   │   │   return f.result()                                                              │
│    91 │                                                                                          │
│    92 │   def _run_once(self):                                                                   │
│    93 │   │   """                                                                                │
│                                                                                                  │
│ /usr/lib/python3.10/asyncio/futures.py:201 in result                                             │
│                                                                                                  │
│   198 │   │   │   raise exceptions.InvalidStateError('Result is not ready.')                     │
│   199 │   │   self.__log_traceback = False                                                       │
│   200 │   │   if self._exception is not None:                                                    │
│ ❱ 201 │   │   │   raise self._exception.with_traceback(self._exception_tb)                       │
│   202 │   │   return self._result                                                                │
│   203 │                                                                                          │
│   204 │   def exception(self):                                                                   │
│                                                                                                  │
│ /usr/lib/python3.10/asyncio/tasks.py:232 in __step                                               │
│                                                                                                  │
│   229 │   │   │   if exc is None:                                                                │
│   230 │   │   │   │   # We use the `send` method directly, because coroutines                    │
│   231 │   │   │   │   # don't have `__iter__` and `__next__` methods.                            │
│ ❱ 232 │   │   │   │   result = coro.send(None)                                                   │
│   233 │   │   │   else:                                    

In [37]:
# define a guidance program that adapts a proverb
program = guidance("""{{person}} is a {{gen 'description1' stop="."}} and a {{gen 'description2' stop="."}}""")

# execute the program on a specific proverb
executed_program = program(
  person="Elon Musk"
)
print(executed_program)

Stop program Elon Musk is a genius and a visionary

Elon Musk is a genius and a visionary
